### Part 1: AWS S3 & Sourcing Datasets
Republish this open dataset in Amazon S3 and share with us a link.
You may run into 403 Forbidden errors as you test accessing this data. There is a way to comply with the BLS data access policies and re-gain access to fetch this data programatically - we have included some hints as to how to do this at the bottom of this README in the Q/A section.

# Required sub-steps to accomplisht this:
1. Set up S3 bucket environmnet
2. Be able to read and publish into S3 env. 
3. troubleshoot error

- Key notes: This is DataLake design. Create a landing-zone where data will be uploaded and parsed under upload date. The goal is just to have a starting point to being bringing in data into s3.

In [1]:
import boto3
import os
from dotenv import load_dotenv

In [2]:
load_dotenv('.env')

True

In [7]:
access_key = os.getenv('AWS_ACCESS_KEY')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')

In [8]:
s3_client = boto3.client(
    's3',
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key
)

In [9]:
response = s3_client.list_buckets()

if 'Buckets' in response:
    buckets = response['Buckets']
    for bucket in buckets:
        print(bucket['Name'])

2021-04-02-ep-website
aws-emr-resources-386175835981-us-east-2
aws-logs-386175835981-us-east-2
bigbatbucket
canvas-bucket-iris-4123
dbdatalocation
ed-exp-cost-and-usage
eddysfistbuck
edwardplatagschoolcap
eplatacapstonedata
eplatacapstoneipynb
qep-sports-betting-bucket
rearc-datalake-bucket
redditdatacollectionwemeta
sagemaker-soln-ddf-js-2ruwg4-386175835981-us-east-1
sagemaker-soln-ddf-js-2seloa-386175835981-us-east-1
sagemaker-soln-ddf-js-2sf3s6-386175835981-us-east-1
sagemaker-soln-ddf-js-44xdya-386175835981-us-east-1
sagemaker-soln-documents-js-4htc2a-us-east-1-386175835981
sagemaker-studio-386175835981-l4ayz3cscdq
sagemaker-studio-386175835981-l9tzph12na
sagemaker-studio-386175835981-zzqac2052o
sagemaker-us-east-2-386175835981
someonesbucket
ss-discord-group-minecraft-bucket


We have access to the s3 bucekt

#### Part 1: AWS S3 & Sourcing Datasets
1. Republish [this open dataset](https://download.bls.gov/pub/time.series/pr/) in Amazon S3 and share with us a link.
    - You may run into 403 Forbidden errors as you test accessing this data. There is a way to comply with the BLS data access policies and re-gain access to fetch this data programatically - we have included some hints as to how to do this at the bottom of this README in the Q/A section.
2. Script this process so the files in the S3 bucket are kept in sync with the source when data on the website is updated, added, or deleted.
    - Don't rely on hard coded names - the script should be able to handle added or removed files.
    - Ensure the script doesn't upload the same file more than once.

Failed to fetch data from https://download.bls.gov/pub/time.series/pr/pr.class
Failed to fetch data from https://download.bls.gov/pub/time.series/pr/pr.contacts
Failed to fetch data from https://download.bls.gov/pub/time.series/pr/pr.data.0.Current
Failed to fetch data from https://download.bls.gov/pub/time.series/pr/pr.data.1.AllData
Failed to fetch data from https://download.bls.gov/pub/time.series/pr/pr.duration
Failed to fetch data from https://download.bls.gov/pub/time.series/pr/pr.footnote
Failed to fetch data from https://download.bls.gov/pub/time.series/pr/pr.measure
Failed to fetch data from https://download.bls.gov/pub/time.series/pr/pr.period
Failed to fetch data from https://download.bls.gov/pub/time.series/pr/pr.seasonal
Failed to fetch data from https://download.bls.gov/pub/time.series/pr/pr.sector
Failed to fetch data from https://download.bls.gov/pub/time.series/pr/pr.series
Failed to fetch data from https://download.bls.gov/pub/time.series/pr/pr.txt


In [1]:
import requests

url = 'https://download.bls.gov/pub/time.series/pr/'
response = requests.get(url)

if response.status_code == 200:
    files = response.text.split('\n')
    for file in files:
        print(file)


In [3]:
print(response)

<Response [403]>


In [1]:
#Create function to fetch all the data from these sources:
#https://download.bls.gov/pub/time.series/pr/pr.class
#https://download.bls.gov/pub/time.series/pr/pr.contacts
#https://download.bls.gov/pub/time.series/pr/pr.data.0.Current
#https://download.bls.gov/pub/time.series/pr/pr.data.1.AllData
#https://download.bls.gov/pub/time.series/pr/pr.duration
#https://download.bls.gov/pub/time.series/pr/pr.footnote
#https://download.bls.gov/pub/time.series/pr/pr.measure
#https://download.bls.gov/pub/time.series/pr/pr.period
#https://download.bls.gov/pub/time.series/pr/pr.seasonal
#https://download.bls.gov/pub/time.series/pr/pr.sector
#https://download.bls.gov/pub/time.series/pr/pr.series
#https://download.bls.gov/pub/time.series/pr/pr.txt


In [5]:
# List of URLs to fetch data from
urls = [
    "https://download.bls.gov/pub/time.series/pr/pr.class",
    "https://download.bls.gov/pub/time.series/pr/pr.contacts",
    "https://download.bls.gov/pub/time.series/pr/pr.data.0.Current",
    "https://download.bls.gov/pub/time.series/pr/pr.data.1.AllData",
    "https://download.bls.gov/pub/time.series/pr/pr.duration",
    "https://download.bls.gov/pub/time.series/pr/pr.footnote",
    "https://download.bls.gov/pub/time.series/pr/pr.measure",
    "https://download.bls.gov/pub/time.series/pr/pr.period",
    "https://download.bls.gov/pub/time.series/pr/pr.seasonal",
    "https://download.bls.gov/pub/time.series/pr/pr.sector",
    "https://download.bls.gov/pub/time.series/pr/pr.series",
    "https://download.bls.gov/pub/time.series/pr/pr.txt"
]



In [6]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [11]:

def generate_urls(base_url):
    # Set the user agent header
    headers = {'User-Agent': 'Mozilla/5.0'}

    # Send a GET request to the URL with the user agent header
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all the <a> tags in the HTML
        links = soup.find_all('a')

        # Extract the href attribute from each <a> tag
        urls = [link.get('href') for link in links if link.get('href')]

        # Prepend the base URL to each URL
        urls = [urljoin(base_url, url) for url in urls]

        return urls
    else:
        print(f"Failed to fetch HTML from {base_url}")
        return []

# Example usage
base_url = 'https://download.bls.gov/pub/time.series/pr/'
urls_list = generate_urls(base_url)
print(urls_list)

['https://download.bls.gov/pub/time.series/', 'https://download.bls.gov/pub/time.series/pr/pr.class', 'https://download.bls.gov/pub/time.series/pr/pr.contacts', 'https://download.bls.gov/pub/time.series/pr/pr.data.0.Current', 'https://download.bls.gov/pub/time.series/pr/pr.data.1.AllData', 'https://download.bls.gov/pub/time.series/pr/pr.duration', 'https://download.bls.gov/pub/time.series/pr/pr.footnote', 'https://download.bls.gov/pub/time.series/pr/pr.measure', 'https://download.bls.gov/pub/time.series/pr/pr.period', 'https://download.bls.gov/pub/time.series/pr/pr.seasonal', 'https://download.bls.gov/pub/time.series/pr/pr.sector', 'https://download.bls.gov/pub/time.series/pr/pr.series', 'https://download.bls.gov/pub/time.series/pr/pr.txt']


In [12]:
urls_list == urls

False

In [13]:
#lets remove the first one, as that is the base url
urls.pop(0)

'https://download.bls.gov/pub/time.series/pr/pr.class'

In [22]:
import requests
import time

def fetch_data(urls):
    data = {}
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    for url in urls:
        # Send a GET request to the URL with the user agent header
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            # Get the response content as text
            page_source = response.text
            data[url] = page_source
        else:
            print(f"Failed to fetch data from {url}")
        
        # Add a delay between requests
        time.sleep(1)

    return data


In [29]:
import pandas as pd
dataframes = {}

for url in urls:
    data = fetch_data([url])
    dataframe = pd.DataFrame(data[url].split('\n'))
    dataframe = dataframe[0].str.split('\t', expand=True)
    dataframes[url] = dataframe


In [38]:
import pprint
pprint.pprint(dataframes['https://download.bls.gov/pub/time.series/pr/pr.class'])

            0            1              2           3                4
0  class_code   class_text  display_level  selectable  sort_sequence\r
1           3    Employees              0           T              2\r
2           6  All workers              0           T              1\r
3                     None           None        None             None


In [39]:
for url, dataframe in dataframes.items():
    dataframe.columns = dataframe.iloc[0]
    dataframe = dataframe[1:]
    dataframes[url] = dataframe

In [ ]:
import os

In [46]:
import os

def save_dataframes_as_csv(dataframes):
    # Create the "data" directory if it doesn't exist
    if not os.path.exists("data"):
        os.makedirs("data")

    for url, dataframe in dataframes.items():
        # Modify the dataset name for the CSV file
        dataset_name = url.split("/")[-1].replace(".", "_") + ".csv"
        # Save the dataframe as a CSV file in the "data" directory
        dataframe.to_csv(os.path.join("data", dataset_name), index=False)

# Example usage
save_dataframes_as_csv(dataframes)


In [48]:
import pandas as pd
import os

# Get the list of CSV files in the "data" directory
csv_files = [file for file in os.listdir("data") if file.endswith(".csv")]

# Read each CSV file and display its contents
for file in csv_files:
    file_path = os.path.join("data", file)
    df = pd.read_csv(file_path)
    print(f"Dataset: {file}")
    print(df.head())  # Display the first few rows of the dataset
    print("\n")


Dataset: pr_series.csv
   series_id          sector_code  class_code  measure_code  duration_code  \
0  PRS30006011             3000.0         6.0           1.0            1.0   
1  PRS30006012             3000.0         6.0           1.0            2.0   
2  PRS30006013             3000.0         6.0           1.0            3.0   
3  PRS30006021             3000.0         6.0           2.0            1.0   
4  PRS30006022             3000.0         6.0           2.0            2.0   

  seasonal base_year  footnote_codes  begin_year begin_period  end_year  \
0        S         -             NaN      1988.0          Q01    2023.0   
1        S         -             NaN      1987.0          Q02    2023.0   
2        S      2017             NaN      1987.0          Q01    2023.0   
3        S         -             NaN      1988.0          Q01    2023.0   
4        S         -             NaN      1987.0          Q02    2023.0   

  end_period  
0        Q03  
1        Q03  
2        Q03

In [1]:
from utils.storage_class import S3Wrapper
import os
import requests

# S3 bucket and landing zone details
bucket_name = 'rearc-datalake-bucket'
landing_zone_prefix = 'landing-zone/'

# Create an instance of the s3wrapper class
s3 = S3Wrapper()

AttributeError: 'S3Wrapper' object has no attribute 'list_buckets'